# 📊 Data Preprocessing & Quality Control with Google Drive

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kstawiski/OmicSelector2/blob/main/examples/04_data_preprocessing.ipynb)

Learn how to prepare your own biomarker data for analysis with full Google Drive integration.

**What You'll Learn:**
- 📁 Load data from multiple formats (CSV, Excel, TSV) stored in Google Drive
- 🧹 Quality control and data cleaning
- 📈 Normalization methods (TPM, Log, Z-score)
- 🔍 Filter low-quality features
- ✂️ Train/test/validation splitting
- ⚖️ Handle imbalanced datasets
- 💾 Save processed data back to Drive

**Estimated Time**: 20-25 minutes  
**Prerequisites**: Basic knowledge of Python and pandas

---

## 🚀 Setup: Install & Mount Drive

In [ ]:
# Install OmicSelector2
!pip install -q git+https://github.com/kstawiski/OmicSelector2.git
!pip install -q openpyxl xlrd  # For Excel support

print("✅ Installation complete!")

In [ ]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=False)

# Setup project directories
BASE_DIR = '/content/drive/MyDrive/OmicSelector2'
os.makedirs(f'{BASE_DIR}/data/raw', exist_ok=True)
os.makedirs(f'{BASE_DIR}/data/processed', exist_ok=True)
os.makedirs(f'{BASE_DIR}/data/qc_reports', exist_ok=True)
os.makedirs(f'{BASE_DIR}/plots/preprocessing', exist_ok=True)

print(f"✅ Drive mounted: {BASE_DIR}")
print("\n📁 Directory structure:")
print(f"  • Raw data: {BASE_DIR}/data/raw/")
print(f"  • Processed data: {BASE_DIR}/data/processed/")
print(f"  • QC reports: {BASE_DIR}/data/qc_reports/")
print(f"  • Plots: {BASE_DIR}/plots/preprocessing/")

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries loaded!")

## 📥 Step 1: Load Data from Google Drive

We'll create sample data, but you can upload your own to `data/raw/` in Drive.

In [ ]:
# For demo: Create sample data and save to Drive
# (Replace this with your own data upload)

print("🧬 Creating sample gene expression dataset...\n")

np.random.seed(42)

# Generate raw counts (simulating RNA-seq)
n_samples = 150
n_genes = 1000

# Simulate count data with some noise
base_counts = np.random.negative_binomial(n=10, p=0.3, size=(n_samples, n_genes))

# Create DataFrame
sample_ids = [f"SAMPLE_{i:03d}" for i in range(n_samples)]
gene_ids = [f"GENE_{i:04d}" for i in range(n_genes)]
X_raw = pd.DataFrame(base_counts, columns=gene_ids, index=sample_ids)

# Clinical data
clinical = pd.DataFrame({
    'sample_id': sample_ids,
    'age': np.random.randint(40, 80, n_samples),
    'gender': np.random.choice(['Male', 'Female'], n_samples),
    'stage': np.random.choice(['I', 'II', 'III', 'IV'], n_samples),
    'response': np.random.binomial(1, 0.4, n_samples),
    'batch': np.random.choice(['Batch_A', 'Batch_B'], n_samples)
})

# Add some missing values (realistic scenario)
missing_mask = np.random.random(X_raw.shape) < 0.01  # 1% missing
X_raw = X_raw.mask(missing_mask)

# Save to Drive
raw_data_path = f"{BASE_DIR}/data/raw/gene_expression_raw.csv"
clinical_path = f"{BASE_DIR}/data/raw/clinical_data.csv"

X_raw.to_csv(raw_data_path)
clinical.to_csv(clinical_path, index=False)

print(f"✅ Sample data created and saved to Drive:")
print(f"   • Expression data: {raw_data_path}")
print(f"   • Clinical data: {clinical_path}")
print(f"\n📊 Dataset info:")
print(f"   • Samples: {X_raw.shape[0]}")
print(f"   • Genes: {X_raw.shape[1]}")
print(f"   • Missing values: {X_raw.isna().sum().sum()}")

# Display previews
print("\n📋 Gene expression preview:")
display(X_raw.iloc[:5, :5])

print("\n📋 Clinical data preview:")
display(clinical.head())

### 💡 How to Use Your Own Data

**Option 1: Upload via Colab Interface**
```python
from google.colab import files
uploaded = files.upload()  # Click to upload
# Then move to Drive:
!mv your_file.csv /content/drive/MyDrive/OmicSelector2/data/raw/
```

**Option 2: Upload Directly to Drive**
1. Open Google Drive in browser
2. Navigate to `OmicSelector2/data/raw/`
3. Drag and drop your files
4. Refresh this notebook and load the files

**Option 3: Share from Another Drive**
```python
# If data is shared with you, mount and access
shared_data_path = '/content/drive/Shareddrives/YourTeam/data.csv'
```

In [ ]:
# Load data from Drive (replace with your file paths)
print("📂 Loading data from Google Drive...\n")

X_raw = pd.read_csv(raw_data_path, index_col=0)
clinical = pd.read_csv(clinical_path)
y = clinical['response'].values

print(f"✅ Data loaded successfully!")
print(f"   • Expression matrix: {X_raw.shape}")
print(f"   • Clinical data: {clinical.shape}")

## 🔍 Step 2: Quality Control Analysis

Assess data quality before preprocessing.

In [ ]:
print("🔍 Quality Control Analysis\n" + "="*60 + "\n")

# Define threshold for missing values (as percentage)
MISSING_THRESHOLD_PCT = 5  # Keep genes/samples with ≤5% missing values

# 1. Missing values
missing_per_gene = X_raw.isna().sum() / len(X_raw) * 100
missing_per_sample = X_raw.isna().sum(axis=1) / X_raw.shape[1] * 100

print(f"📊 Missing Values:")
print(f"   • Total missing: {X_raw.isna().sum().sum():,}")
print(f"   • Genes with >5% missing: {(missing_per_gene > MISSING_THRESHOLD_PCT).sum()}")
print(f"   • Samples with >5% missing: {(missing_per_sample > MISSING_THRESHOLD_PCT).sum()}")

# 2. Zero/low expression genes
zero_counts = (X_raw == 0).sum(axis=0)
low_expr_genes = (zero_counts > len(X_raw) * 0.9).sum()

print(f"\n📊 Expression Levels:")
print(f"   • Genes with >90% zeros: {low_expr_genes}")

# 3. Sample-level QC
lib_sizes = X_raw.sum(axis=1)
print(f"\n📊 Library Sizes (total counts per sample):")
print(f"   • Mean: {lib_sizes.mean():,.0f}")
print(f"   • Median: {lib_sizes.median():,.0f}")
print(f"   • Min: {lib_sizes.min():,.0f}")
print(f"   • Max: {lib_sizes.max():,.0f}")

# 4. Variance analysis
gene_variances = X_raw.var(axis=0)
low_var_genes = (gene_variances < gene_variances.quantile(0.1)).sum()

print(f"\n📊 Gene Variance:")
print(f"   • Genes in bottom 10% variance: {low_var_genes}")

# Visualize QC metrics
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Missing values heatmap (first 50 genes)
sns.heatmap(X_raw.iloc[:, :50].isna(), cbar=False, ax=axes[0, 0])
axes[0, 0].set_title('Missing Values Pattern (First 50 Genes)')
axes[0, 0].set_xlabel('Genes')
axes[0, 0].set_ylabel('Samples')

# Library size distribution
axes[0, 1].hist(lib_sizes, bins=30, edgecolor='black')
axes[0, 1].set_xlabel('Library Size (Total Counts)')
axes[0, 1].set_ylabel('Number of Samples')
axes[0, 1].set_title('Library Size Distribution')
axes[0, 1].axvline(lib_sizes.median(), color='red', linestyle='--', label='Median')
axes[0, 1].legend()

# Zero count distribution
axes[0, 2].hist(zero_counts, bins=50, edgecolor='black')
axes[0, 2].set_xlabel('Number of Zeros')
axes[0, 2].set_ylabel('Number of Genes')
axes[0, 2].set_title('Zero Count Distribution Across Genes')
axes[0, 2].axvline(len(X_raw) * 0.9, color='red', linestyle='--', label='90% threshold')
axes[0, 2].legend()

# Gene variance distribution
axes[1, 0].hist(np.log10(gene_variances + 1), bins=50, edgecolor='black')
axes[1, 0].set_xlabel('Log10(Variance + 1)')
axes[1, 0].set_ylabel('Number of Genes')
axes[1, 0].set_title('Gene Variance Distribution')

# Sample correlation matrix (sample 20 samples)
sample_indices = np.random.choice(len(X_raw), min(20, len(X_raw)), replace=False)
corr_matrix = X_raw.iloc[sample_indices].T.corr()
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, ax=axes[1, 1], cbar_kws={'label': 'Correlation'})
axes[1, 1].set_title('Sample-Sample Correlation (20 samples)')

# Box plot of top varying genes
top_var_genes = gene_variances.nlargest(10).index
axes[1, 2].boxplot([X_raw[gene].dropna() for gene in top_var_genes], labels=range(1, 11))
axes[1, 2].set_xlabel('Top 10 Variable Genes (rank)')
axes[1, 2].set_ylabel('Expression Level')
axes[1, 2].set_title('Top Variable Genes Distribution')

plt.tight_layout()
qc_plot_path = f"{BASE_DIR}/plots/preprocessing/01_quality_control.png"
plt.savefig(qc_plot_path, dpi=300, bbox_inches='tight')
print(f"\n💾 QC plots saved to: {qc_plot_path}")
plt.show()

# Save QC report
qc_report = pd.DataFrame({
    'metric': [
        'Total samples', 'Total genes', 'Total missing values',
        'Genes with >5% missing', 'Samples with >5% missing',
        'Genes with >90% zeros', 'Low variance genes (bottom 10%)',
        'Mean library size', 'Median library size'
    ],
    'value': [
        X_raw.shape[0], X_raw.shape[1], X_raw.isna().sum().sum(),
        (missing_per_gene > MISSING_THRESHOLD_PCT).sum(), (missing_per_sample > MISSING_THRESHOLD_PCT).sum(),
        low_expr_genes, low_var_genes,
        f"{lib_sizes.mean():.0f}", f"{lib_sizes.median():.0f}"
    ]
})

qc_report_path = f"{BASE_DIR}/data/qc_reports/qc_summary.csv"
qc_report.to_csv(qc_report_path, index=False)
print(f"💾 QC report saved to: {qc_report_path}")

## 🧹 Step 3: Data Cleaning

Remove low-quality samples and genes.

In [ ]:
print("🧹 Data Cleaning\n" + "="*60 + "\n")

X_clean = X_raw.copy()
removed_genes = []
removed_samples = []

# 1. Remove genes with too many missing values
genes_to_keep = missing_per_gene <= MISSING_THRESHOLD_PCT  # Keep genes with ≤5% missing
removed_genes.extend(X_clean.columns[~genes_to_keep].tolist())
X_clean = X_clean.loc[:, genes_to_keep]
print(f"✓ Removed {(~genes_to_keep).sum()} genes with >5% missing values")

# 2. Remove genes with low expression (>90% zeros)
zero_counts_clean = (X_clean == 0).sum(axis=0)
high_zero_genes = zero_counts_clean > len(X_clean) * 0.9
removed_genes.extend(X_clean.columns[high_zero_genes].tolist())
X_clean = X_clean.loc[:, ~high_zero_genes]
print(f"✓ Removed {high_zero_genes.sum()} genes with >90% zeros")

# 3. Remove low variance genes (bottom 10%)
var_threshold = X_clean.var(axis=0).quantile(0.1)
low_var = X_clean.var(axis=0) < var_threshold
removed_genes.extend(X_clean.columns[low_var].tolist())
X_clean = X_clean.loc[:, ~low_var]
print(f"✓ Removed {low_var.sum()} low-variance genes (bottom 10%)")

# 4. Handle remaining missing values (impute with median)
missing_before = X_clean.isna().sum().sum()
if missing_before > 0:
    X_clean = X_clean.fillna(X_clean.median())
    print(f"✓ Imputed {missing_before} remaining missing values with gene medians")

print(f"\n📊 After cleaning:")
print(f"   • Samples: {X_clean.shape[0]} (removed {len(removed_samples)})")
print(f"   • Genes: {X_clean.shape[1]} (removed {len(set(removed_genes))})")
print(f"   • Data completeness: 100%")

# Save cleaned data
clean_data_path = f"{BASE_DIR}/data/processed/gene_expression_cleaned.csv"
X_clean.to_csv(clean_data_path)
print(f"\n💾 Cleaned data saved to: {clean_data_path}")

# Save list of removed genes
removed_genes_df = pd.DataFrame({'gene': list(set(removed_genes))})
removed_genes_path = f"{BASE_DIR}/data/qc_reports/removed_genes.csv"
removed_genes_df.to_csv(removed_genes_path, index=False)
print(f"💾 Removed genes list saved to: {removed_genes_path}")

## 📈 Step 4: Normalization

Apply different normalization methods.

In [ ]:
print("📈 Data Normalization\n" + "="*60 + "\n")

# We'll create multiple normalized versions
normalized_data = {}

# 1. Log2 transformation (common for RNA-seq)
X_log2 = np.log2(X_clean + 1)  # +1 to avoid log(0)
normalized_data['log2'] = X_log2
print("✓ Log2 transformation: log2(x + 1)")

# 2. Z-score normalization (standardization)
scaler_zscore = StandardScaler()
X_zscore = pd.DataFrame(
    scaler_zscore.fit_transform(X_log2),
    columns=X_log2.columns,
    index=X_log2.index
)
normalized_data['zscore'] = X_zscore
print("✓ Z-score normalization: (x - mean) / std")

# 3. Robust scaling (less sensitive to outliers)
scaler_robust = RobustScaler()
X_robust = pd.DataFrame(
    scaler_robust.fit_transform(X_log2),
    columns=X_log2.columns,
    index=X_log2.index
)
normalized_data['robust'] = X_robust
print("✓ Robust scaling: (x - median) / IQR")

# 4. TPM-like normalization (per-sample)
# Total counts per sample = 1 million
X_tpm = X_clean.div(X_clean.sum(axis=1), axis=0) * 1e6
X_tpm_log = np.log2(X_tpm + 1)
normalized_data['tpm_log'] = X_tpm_log
print("✓ TPM-like normalization: (count/total) * 1M, then log2")

print(f"\n📊 Created {len(normalized_data)} normalized versions")

# Visualize normalization effects
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

# Original (cleaned)
sample_gene = X_clean.iloc[:, 0]
axes[0].hist(sample_gene, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Original (Cleaned)')
axes[0].set_xlabel('Expression Level')
axes[0].set_ylabel('Frequency')

# Each normalization method
for idx, (name, data) in enumerate(normalized_data.items(), 1):
    sample_gene_norm = data.iloc[:, 0]
    axes[idx].hist(sample_gene_norm, bins=50, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{name.upper()}')
    axes[idx].set_xlabel('Expression Level')
    axes[idx].set_ylabel('Frequency')

# Hide last subplot if not used
if len(normalized_data) < 5:
    axes[5].axis('off')

plt.suptitle('Effect of Different Normalization Methods (Example Gene)', fontsize=14, fontweight='bold')
plt.tight_layout()

norm_plot_path = f"{BASE_DIR}/plots/preprocessing/02_normalization_comparison.png"
plt.savefig(norm_plot_path, dpi=300, bbox_inches='tight')
print(f"\n💾 Normalization comparison plot saved to: {norm_plot_path}")
plt.show()

# Save all normalized versions
for name, data in normalized_data.items():
    save_path = f"{BASE_DIR}/data/processed/gene_expression_{name}.csv"
    data.to_csv(save_path)
    print(f"💾 Saved {name} normalized data to: {save_path}")

# Use log2 + z-score for further analysis (common choice)
X_normalized = X_zscore
print(f"\n✅ Using Z-score normalized data for downstream analysis")

## ✂️ Step 5: Train/Test/Validation Split

Proper data splitting to prevent overfitting.

In [ ]:
print("✂️ Data Splitting\n" + "="*60 + "\n")

# First split: 60% train, 40% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X_normalized, y,
    test_size=0.4,
    random_state=42,
    stratify=y
)

# Second split: split temp into 20% validation, 20% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

print("📊 Split sizes:")
print(f"   • Training:   {X_train.shape[0]} samples ({X_train.shape[0]/len(X_normalized)*100:.1f}%)")
print(f"   • Validation: {X_val.shape[0]} samples ({X_val.shape[0]/len(X_normalized)*100:.1f}%)")
print(f"   • Test:       {X_test.shape[0]} samples ({X_test.shape[0]/len(X_normalized)*100:.1f}%)")

print("\n📊 Class distribution:")
print(f"   • Training:   {np.bincount(y_train)}")
print(f"   • Validation: {np.bincount(y_val)}")
print(f"   • Test:       {np.bincount(y_test)}")

# Visualize split
fig, ax = plt.subplots(figsize=(10, 6))

splits = ['Training', 'Validation', 'Test']
class_0 = [np.sum(y_train == 0), np.sum(y_val == 0), np.sum(y_test == 0)]
class_1 = [np.sum(y_train == 1), np.sum(y_val == 1), np.sum(y_test == 1)]

x = np.arange(len(splits))
width = 0.35

bars1 = ax.bar(x - width/2, class_0, width, label='Class 0 (Non-responder)', alpha=0.8)
bars2 = ax.bar(x + width/2, class_1, width, label='Class 1 (Responder)', alpha=0.8)

ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Class Distribution Across Data Splits', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(splits)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
split_plot_path = f"{BASE_DIR}/plots/preprocessing/03_data_splits.png"
plt.savefig(split_plot_path, dpi=300, bbox_inches='tight')
print(f"\n💾 Split visualization saved to: {split_plot_path}")
plt.show()

# Save splits to Drive
print("\n💾 Saving data splits to Google Drive...")

splits_dir = f"{BASE_DIR}/data/processed/splits"
os.makedirs(splits_dir, exist_ok=True)

X_train.to_csv(f"{splits_dir}/X_train.csv")
X_val.to_csv(f"{splits_dir}/X_val.csv")
X_test.to_csv(f"{splits_dir}/X_test.csv")

pd.DataFrame({'sample_id': X_train.index, 'response': y_train}).to_csv(f"{splits_dir}/y_train.csv", index=False)
pd.DataFrame({'sample_id': X_val.index, 'response': y_val}).to_csv(f"{splits_dir}/y_val.csv", index=False)
pd.DataFrame({'sample_id': X_test.index, 'response': y_test}).to_csv(f"{splits_dir}/y_test.csv", index=False)

print(f"✅ All splits saved to: {splits_dir}/")
print("\n💡 These splits are ready for feature selection and model training!")

## ⚖️ Step 6: Handle Class Imbalance (Optional)

If your classes are imbalanced, here are some strategies.

In [ ]:
from collections import Counter

print("⚖️ Class Imbalance Analysis\n" + "="*60 + "\n")

# Check imbalance ratio
class_counts = Counter(y_train)
majority_class = max(class_counts, key=class_counts.get)
minority_class = min(class_counts, key=class_counts.get)
imbalance_ratio = class_counts[majority_class] / class_counts[minority_class]

print(f"📊 Training set class distribution:")
print(f"   • Class 0: {class_counts[0]} samples")
print(f"   • Class 1: {class_counts[1]} samples")
print(f"   • Imbalance ratio: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 1.5:
    print(f"\n⚠️ Moderate imbalance detected (ratio > 1.5:1)")
    print("\n💡 Recommended strategies:")
    print("   1. Use class weights in model training")
    print("   2. Use stratified sampling (already done)")
    print("   3. Use balanced metrics (F1, balanced accuracy)")
    
    if imbalance_ratio > 3:
        print("   4. Consider SMOTE (Synthetic Minority Over-sampling)")
        print("   5. Consider undersampling majority class")
    
    # Example: Calculate class weights for sklearn
    from sklearn.utils.class_weight import compute_class_weight
    
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weight_dict = dict(enumerate(class_weights))
    
    print(f"\n✅ Computed class weights: {class_weight_dict}")
    print("   Use these when training: model.fit(X, y, class_weight=class_weight_dict)")
    
    # Save class weights
    class_weights_df = pd.DataFrame({
        'class': list(class_weight_dict.keys()),
        'weight': list(class_weight_dict.values())
    })
    weights_path = f"{BASE_DIR}/data/processed/class_weights.csv"
    class_weights_df.to_csv(weights_path, index=False)
    print(f"\n💾 Class weights saved to: {weights_path}")
else:
    print(f"\n✅ Classes are relatively balanced (ratio < 1.5:1)")
    print("   No special handling needed.")

## 📝 Step 7: Create Processing Summary

Document everything for reproducibility.

In [ ]:
from datetime import datetime

# Create comprehensive summary
summary = f"""
═══════════════════════════════════════════════════════════════
    DATA PREPROCESSING SUMMARY - OmicSelector2
═══════════════════════════════════════════════════════════════

Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Google Drive Location: {BASE_DIR}

ORIGINAL DATA
───────────────────────────────────────────────────────────────
Samples:              {X_raw.shape[0]}
Genes:                {X_raw.shape[1]}
Missing values:       {X_raw.isna().sum().sum():,}

DATA CLEANING
───────────────────────────────────────────────────────────────
Removed samples:      {len(removed_samples)}
Removed genes:        {len(set(removed_genes))}
  • >5% missing:      {(missing_per_gene > MISSING_THRESHOLD_PCT).sum()}
  • >90% zeros:       {low_expr_genes}
  • Low variance:     {low_var.sum()}

FINAL CLEANED DATA
───────────────────────────────────────────────────────────────
Samples:              {X_clean.shape[0]}
Genes:                {X_clean.shape[1]}
Missing values:       0
Completeness:         100%

NORMALIZATION
───────────────────────────────────────────────────────────────
Methods applied:      {', '.join(normalized_data.keys())}
Selected method:      Z-score (standardization)

DATA SPLITS
───────────────────────────────────────────────────────────────
Training:             {X_train.shape[0]} samples ({X_train.shape[0]/len(X_normalized)*100:.1f}%)
Validation:           {X_val.shape[0]} samples ({X_val.shape[0]/len(X_normalized)*100:.1f}%)
Test:                 {X_test.shape[0]} samples ({X_test.shape[0]/len(X_normalized)*100:.1f}%)

Class distribution (train):
  • Class 0:          {np.sum(y_train == 0)} samples
  • Class 1:          {np.sum(y_train == 1)} samples
  • Imbalance ratio:  {imbalance_ratio:.2f}:1

FILES SAVED TO GOOGLE DRIVE
───────────────────────────────────────────────────────────────
Raw data:             {BASE_DIR}/data/raw/
Cleaned data:         {BASE_DIR}/data/processed/gene_expression_cleaned.csv
Normalized data:      {BASE_DIR}/data/processed/gene_expression_*.csv
Data splits:          {BASE_DIR}/data/processed/splits/
QC reports:           {BASE_DIR}/data/qc_reports/
Plots:                {BASE_DIR}/plots/preprocessing/

NEXT STEPS
───────────────────────────────────────────────────────────────
✓ Data is cleaned and ready for analysis
✓ Multiple normalized versions available
✓ Proper train/val/test splits created
✓ All files persisted in Google Drive

→ Continue with feature selection (01_basic_feature_selection.ipynb)
→ Or try stability selection (05_stability_selection.ipynb)

═══════════════════════════════════════════════════════════════
"""

print(summary)

# Save summary
summary_path = f"{BASE_DIR}/data/processed/preprocessing_summary.txt"
with open(summary_path, 'w') as f:
    f.write(summary)

print(f"\n💾 Summary saved to: {summary_path}")
print(f"\n✅ All preprocessing complete! Your data is ready in Google Drive.")
print(f"\n📂 Access your files at: {BASE_DIR}")

## 🎓 Summary

### What You've Learned

✅ **Data Loading**: Import data from Google Drive in various formats  
✅ **Quality Control**: Assess data quality with comprehensive metrics  
✅ **Data Cleaning**: Remove low-quality samples and features  
✅ **Normalization**: Apply multiple normalization strategies  
✅ **Data Splitting**: Create proper train/val/test splits  
✅ **Imbalance Handling**: Strategies for imbalanced datasets  
✅ **Drive Integration**: Save all results persistently to Google Drive  

### Your Google Drive Structure

```
MyDrive/OmicSelector2/
├── data/
│   ├── raw/                    # Original data
│   ├── processed/              # Cleaned & normalized data
│   │   ├── splits/             # Train/val/test splits
│   │   └── *.csv               # Various normalized versions
│   └── qc_reports/             # Quality control reports
└── plots/
    └── preprocessing/           # QC and normalization plots
```

### Best Practices Checklist

✅ Always perform QC before analysis  
✅ Document removed samples/features  
✅ Try multiple normalization methods  
✅ Use stratified splitting for classification  
✅ Keep validation and test sets truly independent  
✅ Save intermediate results to Drive  
✅ Create reproducible reports  

### 📚 Next Tutorials

Now that your data is preprocessed, continue with:

1. **[01_basic_feature_selection.ipynb](01_basic_feature_selection.ipynb)** - Select important biomarkers
2. **[05_stability_selection.ipynb](05_stability_selection.ipynb)** - Robust feature selection
3. **[07_complete_workflow.ipynb](07_complete_workflow.ipynb)** - Full analysis pipeline

### 💡 Tips

- **Share with Team**: Share your `OmicSelector2` Drive folder with collaborators
- **Version Control**: Keep dated versions of important files
- **Documentation**: Always save processing summaries
- **Backups**: Drive automatically backs up, but export critical results

---

**Your data is now ready for biomarker discovery! 🧬🔬**